In [1]:
import os
import rasterio
from rasterio.merge import merge
import glob
import cProfile
import io
import os
import requests
import pstats

import numpy as np

import functions

# FOR THE POINT CLOUDS

In [4]:
def download_files(tile_name, sub_tiles):
    base_url = "https://geotiles.citg.tudelft.nl/AHN4_T"
    folder = f"../data/{tile_name}"
    
    if not os.path.exists(folder):
        os.makedirs(folder)

    for sub_tile in sub_tiles:
        sub_tile_str = f"_{sub_tile:02}" 
        url = f"{base_url}/{tile_name}{sub_tile_str}.LAZ"
        filename = f"{tile_name}{sub_tile_str}.LAZ"
        file_path = os.path.join(folder, filename)

        if os.path.exists(file_path):
            print(f"File {file_path} already exists, skipping download.")
            continue

        try:
            response = requests.get(url)
            response.raise_for_status()  # Ensure successful request
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded and saved {file_path}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")

In [5]:
def create_sub_tiles_list(sub_tiles):
    if isinstance(sub_tiles, str) and "-" in sub_tiles:
        start, end = map(int, sub_tiles.split('-'))
        return list(range(start, end + 1))  # inclusive range
    elif isinstance(sub_tiles, list):
        return sub_tiles
    else:
        raise ValueError("Invalid format. Provide a list of numbers or a range as '1-25'.")

In [6]:
download_files('25DN2', '1-2')

KeyboardInterrupt: 

# for the dsm and dtm

In [2]:
def merge_tif_files(input_folder, output_file, nodata_value=-9999):
    tif_files = glob.glob(os.path.join(input_folder, '*.TIF'))

    if not tif_files:
        print("No TIF files found.")
        return

    src_files_to_mosaic = []

    for tif_file in tif_files:
        src = rasterio.open(tif_file)
        src_files_to_mosaic.append(src)

    mosaic, out_transform = merge(src_files_to_mosaic)

    # Replace existing nodata values with new nodata value if necessary
    for src in src_files_to_mosaic:
        if 'nodata' in src.meta:
            mosaic[mosaic == src.nodata] = nodata_value

    out_meta = src_files_to_mosaic[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform,
        "count": mosaic.shape[0],  # Band count
        "nodata": nodata_value  # Set the nodata value
    })

    # Write the merged file to the output path
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

    print(f"Merged {len(tif_files)} TIF files into {output_file}")

In [3]:
input_folder = "data/DSM"
output_file = "data/DSM_ams.TIF"

In [4]:
merge_tif_files(input_folder, output_file)

Merged 14 TIF files into data/DSM_ams.TIF


In [6]:
input_folder = "data/DTM"
output_file = "data/DTM_ams.TIF"
merge_tif_files(input_folder, output_file)

Merged 14 TIF files into data/DTM_ams.TIF
